<a href="https://colab.research.google.com/github/NoemiRyf/ML2_Project/blob/main/Trash_Sorting_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary libraries 

# Neuer Abschnitt

In [ ]:
!ls


sample_data


In [ ]:
%cd /content

/content


In [ ]:
!ls

gggg  sample_data


In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
!git clone https://github.com/garythung/trashnet.git



Cloning into 'trashnet'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 45 (delta 6), reused 0 (delta 0), pack-reused 33
Unpacking objects: 100% (45/45), 40.64 MiB | 13.67 MiB/s, done.


In [ ]:
%cd /content/trashnet/data


/content/trashnet/data


In [ ]:
import zipfile

zip_path = '/content/trashnet/data/dataset-resized.zip'
destination_folder = '/content/trashnet/data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)


In [ ]:
!python resize.py


Define image data generator for training and validation data

In [ ]:
# Define image data generator for training and validation data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load your dataset
train_set = train_datagen.flow_from_directory('/content/trashnet/data/dataset-resized',
                                              target_size=(64, 64),
                                              batch_size=32,
                                              class_mode='categorical')

Found 2527 images belonging to 6 classes.


Build Model

In [ ]:
# Build your model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=6, activation='softmax'))  # 3 units for 3 classes: recyclable, compostable, or trash


Compile Model

In [ ]:
# Compile your model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Train Model

In [ ]:
# Train your model
model.fit(train_set, epochs=25)

Epoch 1/25
79/79 [==============================] - 25s 296ms/step - loss: 1.7412 - accuracy: 0.3320
Epoch 2/25
79/79 [==============================] - 23s 294ms/step - loss: 1.3438 - accuracy: 0.4832
Epoch 3/25
79/79 [==============================] - 23s 296ms/step - loss: 1.2291 - accuracy: 0.5323
Epoch 4/25
79/79 [==============================] - 26s 324ms/step - loss: 1.1804 - accuracy: 0.5710
Epoch 5/25
79/79 [==============================] - 23s 296ms/step - loss: 1.1289 - accuracy: 0.5706
Epoch 6/25
79/79 [==============================] - 24s 301ms/step - loss: 1.0806 - accuracy: 0.6086
Epoch 7/25
79/79 [==============================] - 22s 282ms/step - loss: 1.0465 - accuracy: 0.6189
Epoch 8/25
79/79 [==============================] - 23s 285ms/step - loss: 0.9730 - accuracy: 0.6462
Epoch 9/25
79/79 [==============================] - 24s 306ms/step - loss: 0.9576 - accuracy: 0.6407
Epoch 10/25
79/79 [==============================] - 22s 277ms/step - loss: 0.9390 - accura

Save Model

In [ ]:
# Save your model
model.save('/content/trashnet/trash_classifier.h5')

from google.colab import files

# Download the model file
files.download('/content/trashnet/trash_classifier.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from PIL import Image
import numpy as np

# Load and preprocess the test image
test_image_path = '/content/trashnet/test_images/glass.jpg'
test_image = Image.open(test_image_path)
target_size = (64, 64)
test_image = test_image.resize(target_size)
test_image = np.array(test_image) / 255.0
test_image = np.expand_dims(test_image, axis=0)

# Make predictions on the test image
predictions = model.predict(test_image)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])

# Map the predicted class index to the corresponding class label
class_labels = train_set.class_indices
predicted_class_label = {v: k for k, v in class_labels.items()}[predicted_class_index]

# Print the predicted class label
print('Predicted class:', predicted_class_label)


FileNotFoundError: ignored

In [ ]:
import os

content_folder = '/content/trashnet'
for root, dirs, files in os.walk(content_folder):
    for file in files:
        print(os.path.join(root, file))


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
/content/trashnet/data/.git/objects/a6/3a2c41064d826cd085be2ac56e6603d983109b
/content/trashnet/data/.git/objects/a6/da70f05fdb8b367ecba16741ef301220d45852
/content/trashnet/data/.git/objects/a6/bee14169099be687c4779d6a4710fd94a7f4a7
/content/trashnet/data/.git/objects/a6/41f3ffda04a8dd6976f0266b9113c7caaafa28
/content/trashnet/data/.git/objects/a6/af863c4b6a3cfd919e588c3e14c864cf9b1892
/content/trashnet/data/.git/objects/a6/8ecd25f82eade697a6efc43c090f8818b6b026
/content/trashnet/data/.git/objects/a6/a2e0fba3409dbf2c173a8c2e8ce3c290fe77db
/content/trashnet/data/.git/objects/a6/cef4a0e91b23d1695205b6f4d732a64d33179a
/content/trashnet/data/.git/objects/36/962bcb4ce308146bc306ae47a115fc3ff5b287
/content/trashnet/data/.git/objects/36/bd734db266559f6182bce41d4c91e6eb636db4
/content/trashnet/data/.git/objects/36/8f1da170df80a29101bcc2ad0b0d11ff3fa3d0
/content/trashnet/data/.git/objects/36/b6d080f82c868ff9e0eb9cc93aef3c0d7bac

In [ ]:
import shutil
import os
import zipfile

source_folder = '/content/trashnet'  # Pfad zum Ordner, den Sie herunterladen möchten
zip_path = '/content/trashnet.zip'  # Pfad zum Zip-Archiv

# Zip-Archiv erstellen
shutil.make_archive('/content/trashnet', 'zip', source_folder)

# Zip-Archiv in den Zielordner verschieben
shutil.move('/content/trashnet.zip', zip_path)

# Datei herunterladen
from google.colab import files
files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **NLP**

In [ ]:
from tensorflow.keras.models import load_model

image_model_path = '/content/trashnet/trash_classifier.h5'
image_model = load_model(image_model_path)

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.9 MB/s eta 0:00:00


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

nlp_model_name = 'gpt2'  # Choose the NLP model you want to use
nlp_model = GPT2LMHeadModel.from_pretrained(nlp_model_name)
tokenizer = GPT2Tokenizer.from_pretrained(nlp_model_name)


In [31]:
from PIL import Image
import numpy as np

image_path = '/content/trashnet/data/dataset-resized/cardboard/cardboard1.jpg'  # Replace with the path to your image
target_size = (64, 64)  # Same target size used during training the image model

image = Image.open(image_path)
image = image.resize(target_size)
image = np.array(image) / 255.0
image = np.expand_dims(image, axis=0)

In [32]:

predicted_class_index = np.argmax(image_model.predict(image), axis=1)[0]

1/1 [==============================] - 0s 203ms/step


In [38]:
predicted_class_label = {0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}

predicted_class = predicted_class_label[predicted_class_index]

print(f"This item is {predicted_class}.")

def get_disposal_instruction(item_class):
    # Define disposal instructions for each class
    disposal_instructions = {
        'cardboard': 'Please recycle this item in the designated cardboard recycling bin.',
        'glass': 'Please place this item in the glass recycling container.',
        'metal': 'Recycle this item in the metal recycling bin if possible.',
        'paper': 'Dispose of this item in the paper recycling bin.',
        'plastic': 'Recycle this item in the plastic recycling container.',
        'trash': 'Please discard this item in the regular trash bin.'
    }
    return disposal_instructions.get(item_class, 'No disposal instructions available.')

disposal_instruction = get_disposal_instruction(predicted_class)
print(f"To dispose of this item properly, follow these instructions:\n{disposal_instruction}")


This item is cardboard.
To dispose of this item properly, follow these instructions:
Please recycle this item in the designated cardboard recycling bin.
